In [36]:
import numpy as np
import cv2
import yaml
from hsi_dataset_api import HsiDataCropper, HsiDataset
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

In [4]:
dataset = HsiDataset('/mnt/data/cropped_hsi_data')

/home/rustam/anaconda3/lib/python3.7/site-packages/hsi_dataset_api/Dataset.py:56: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(f)


Let's work only with specters that contain 'potato' class

In [22]:
specters_with_potato = []
for data_point in dataset.data_iterator(opened=False):
    specter = data_point.hsi
    mask = data_point.mask
    meta = data_point.meta
    with open(meta, 'r') as f:
        meta = yaml.load(f)
    data_point.meta = meta
    if 'potato' in meta['classes']:
        specters_with_potato.append(data_point)

/home/rustam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [40]:
len(specters_with_potato)

80

This array contains shuffled indices from 0 to length of specters_with_potato

In [17]:
choose_array = np.random.permutation(len(specters_with_potato))

In [41]:
xtrain, xtest, ytrain, ytest = [], [], [], []

In the following cells we create train and test data by taking 20 specters for each sampling.

From each specter we take 1% of vectors randomly.

In [42]:
for idx in choose_array[:20]:
    data_point = specters_with_potato[idx]
    specter = np.load(data_point.hsi)
    mask = cv2.imread(data_point.mask)[..., 0]
    meta = data_point.meta

    for i in range(int(512 * 512 * 0.01)):
        y = np.random.randint(0, 512)
        x = np.random.randint(0, 512)
        vec = specter[::, y, x]

        xtrain.append(vec)
        ytrain.append(mask[y, x])

In [43]:
for idx in choose_array[20:40]:
    data_point = specters_with_potato[idx]
    specter = np.load(data_point.hsi)
    mask = cv2.imread(data_point.mask)[..., 0]
    meta = data_point.meta

    for i in range(int(512 * 512 * 0.01)):
        y = np.random.randint(0, 512)
        x = np.random.randint(0, 512)
        vec = specter[::, y, x]

        xtest.append(vec)
        ytest.append(mask[y, x])

Let's train simple multilayer perceptron

In [44]:
clf = MLPClassifier(random_state=0, max_iter=10_000)
clf.fit(xtrain, ytrain)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=10000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=0, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [45]:
preds = clf.predict(xtest)

In [46]:
print(classification_report(ytest, preds))

              precision    recall  f1-score   support

           0       0.94      0.93      0.94     18916
           1       0.94      0.77      0.85     32516
           2       0.09      0.62      0.15       988

    accuracy                           0.82     52420
   macro avg       0.66      0.77      0.64     52420
weighted avg       0.93      0.82      0.87     52420

